In [ ]:
#!/usr/bin/env python

from pathlib import Path   
from random import randint
from http.server import BaseHTTPRequestHandler, HTTPServer
import socketserver,subprocess,os,requests,json,time,shutil

class S(BaseHTTPRequestHandler):
    def _set_headers(self):
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()

    def do_GET(self):
        self._set_headers()
        self.wfile.write("<html><body><h1>hi!</h1></body></html>")

    def do_HEAD(self):
        self._set_headers()
    
    def do_DELETE (self):
        self._set_headers()
    
    def do_PATCH (self):
        self._set_headers()
        
    def do_POST(self):
        # Doesn't do anything with posted data
        flinkEndpoint="138.4.7.94:8082"
        entryClass=""
        content_length = int(self.headers['Content-Length']) # <--- Gets the size of data
        post_data = self.rfile.read(content_length) # <--- Gets the data itself
        print(post_data)
        ##Call function to generate cep code then call to execute maven, teh code must be in the 
        #current directory
        directory=self.execute_maven()
        jarId=self.upload_jar(flinkEndpoint,directory)
        jobId=self.run_job(jarId,flinkEndpoint)
        self.delete_jar(jarId,flinkEndpoint)
        self._set_headers()

    def execute_maven(self):
        os.chdir('./')
    
        shutil.copytree('./cep', './cep'+prefix)
        mypath = './cep'+prefix
        os.chdir(mypath)
        p = subprocess.Popen(["mvn", "package"], stdout=subprocess.PIPE)
        output, err = p.communicate()
        os.chdir("..")
        print (output)
        return mypath
    
    def upload_jar(self,flinkEndpoint,directory):
        mypath = directory+'/target'
        os.chdir(mypath)
        files = os.listdir('./')
        jarName = ""
        for name in files:
            if 'cep' in name and 'original' not in name:
                jarName = name
                print(name)
        FLINK_ENDPOINT = "http://"+flinkEndpoint+"/jars/upload"
        file_list = [  
        ('jarfile', (jarName, open(jarName, 'rb'), mypath))]
        r = requests.post(url = FLINK_ENDPOINT,  files=file_list) 
        pastebin_url = json.loads(r.text) 
        args=pastebin_url.get("filename").split("/")
        jarId=args[len(args)-1]
        print("About Uploaded Jar:%s"%pastebin_url)
        os.chdir('../..')
        shutil.rmtree('./'+directory, ignore_errors=True)
        return jarId
   
    def run_job(self,jarId,flinkEndpoint):
        FLINK_ENDPOINT = "http://"+flinkEndpoint+"/jars/"+jarId+"/run?allowNonRestoredState=true"
        r = requests.post(url = FLINK_ENDPOINT) 
        pastebin_url = json.loads(r.text)
        jobID=pastebin_url.get("id")
        print("About the running Job:%s"%pastebin_url)    
        return jobID  
    
    def delete_jar(self,jarId,flinkEndpoint):
        FLINK_ENDPOINT = "http://"+flinkEndpoint+"/jars/"+jarId
        r = requests.delete(url = FLINK_ENDPOINT)
    
    def kill_job(self,jobId,flinkEndpoint):
        FLINK_ENDPOINT = "http://"+flinkEndpoint+"/jobs/"+jobId
        requests.patch(url = FLINK_ENDPOINT)
        
def run(server_class=HTTPServer, handler_class=S, port=8092):
    server_address = ('', port)
    httpd = server_class(server_address, handler_class)
    print ('Starting httpd...')
    httpd.serve_forever()

if __name__ == "__main__":
    from sys import argv

    if len(argv) == 2:
        run(port=int(argv[1]))
    else:
        run()

Starting httpd...
b'{    "rule": {        "type": "COUNT_POLICY",        "params": {            "numMaxEvents": 200,            "eventWindow": 15000,             }    },    "punishment": {        "type":  "UNSUBSCRIBE"      }}'
b"[\x1bINFO\x1b[m] Scanning for projects...\n[\x1bINFO\x1b[m] \n[\x1bINFO\x1b[m] \x1b------------------------------------------------------------------------\x1b[m\n[\x1bINFO\x1b[m] \x1bBuilding orion.flink.cep 1.0-SNAPSHOT\x1b[m\n[\x1bINFO\x1b[m] \x1b------------------------------------------------------------------------\x1b[m\n[\x1bINFO\x1b[m] \n[\x1bINFO\x1b[m] \x1b--- \x1bbuild-helper-maven-plugin:1.7:add-source\x1b[m \x1b(add-source)\x1b[m @ \x1borion.flink.cep\x1b ---\x1b[m\n[\x1bINFO\x1b[m] Source directory: /home/amunoz/Desktop/fiware-usage-control/ptp/docker/cep1554740929.3718746/src/main/scala added.\n[\x1bINFO\x1b[m] \n[\x1bINFO\x1b[m] \x1b--- \x1bmaven-resources-plugin:2.6:resources\x1b[m \x1b(default-resources)\x1b[m @ \x1borion.flink.cep\x1b ---\x

About the running Job:{'jobid': '492048a9784bda5fd6bb27aa89a218bc'}


127.0.0.1 - - [08/Apr/2019 18:28:56] "POST / HTTP/1.1" 200 -


In [8]:
import json
def kill_job(self,jobId,flinkEndpoint):
    FLINK_ENDPOINT = "http://"+flinkEndpoint+"/jobs/"+jobId
    requests.patch(url = FLINK_ENDPOINT)
    
kill_job("","5bef5df0b3920a922253faa21d2d5563","138.4.7.94:8082")

About Kill Job:{}


In [103]:

def write_program(self, code):
    f = open("./cep/src/main/scala/org.fiware.cosmos.orion.flink.cep/CEPMonitoring.scala", "r+")
    print ("Name of the file: ", f.name)
    lines= f.readlines()
    for x in lines:
        part1 = part1 + x
        count = count + 1
        if '// TODO' in x:
            break
    count = 0
    for x in lines:
        count = count + 1
        if count > 42:
            part2 = part2 + x             
    f.close()
    w = open("./cep/src/main/scala/org.fiware.cosmos.orion.flink.cep/CEPMonitoring.scala", "w+")
    w.write(part1)
    w.close()
    u = open("./cep/src/main/scala/org.fiware.cosmos.orion.flink.cep/CEPMonitoring.scala", "a")
    u.write(code)
    u.write(part2)
    u.close()



/home/amunoz/Desktop/fiware-usage-control/ptp/docker
Name of the file:  ./cep/src/main/scala/org.fiware.cosmos.orion.flink.cep/CEPMonitoring.scala


In [104]:
    def write_program(self, code):
        print("xxxxxx")
        print(code)
        f = open("./cep/src/main/scala/org.fiware.cosmos.orion.flink.cep/CEPMonitoring.scala", "r+")
        print ("Name of the file: ", f.name)
        part1 = ""
        part2 = ""
        count = 0
        lines = f.readlines()
        for x in lines:
            part1 = part1 + x
            if '// TODO' in x:
                break
        for x in lines:
            count = count + 1
            if count >= 42:
                part2 = part2 + x             
        f.close()
        w = open("./cep/src/main/scala/org.fiware.cosmos.orion.flink.cep/CEPMonitoring.scala", "w+")
        w.write(part1)
        w.close()
        u = open("./cep/src/main/scala/org.fiware.cosmos.orion.flink.cep/CEPMonitoring.scala", "a")
        print("aaaaaaaaaaaaaaaaaaaaaaaaaaaaaa<")
        print(code)
        u.write(code)
        u.write(part2)
        u.close()

In [ ]:
os.path.dirname(os.path.abspath(__file__))


In [2]:
 from pathlib import Path    

In [6]:
import requests 
import json,os



mypath = Path().absolute()/'cep'/'target'
print('Absolute path : {}'.format(mypath))
files = os.listdir(mypath)
for name in files:
    print(name)
        
def upload_job(flinkEndpoint):
    
    mypath = Path().absolute()/'cep'/'target'
    FLINK_ENDPOINT = "http://"+flinkEndpoint+"/jars/upload"
    file_list = [  
    ('jarfile', ('orion.flink.cep-1.0-SNAPSHOT.jar', open('orion.flink.cep-1.0-SNAPSHOT.jar', 'rb'), mypath))]
    r = requests.post(url = FLINK_ENDPOINT,  files=file_list) 
    pastebin_url = json.loads(r.text) 
    args=pastebin_url.get("filename").split("/")
    jarId=args[len(args)-1]
    print("About Uploaded Jar:%s"%pastebin_url)

upload_job("138.4.7.94:8082")

Absolute path : /home/amunoz/Desktop/fiware-usage-control/ptp/docker/cep/target
orion.flink.cep-1.0-SNAPSHOT.jar
classes
maven-status
original-orion.flink.cep-1.0-SNAPSHOT.jar
classes.912030222.timestamp
maven-archiver


FileNotFoundError: [Errno 2] No such file or directory: 'orion.flink.cep-1.0-SNAPSHOT.jar'

In [4]:
import time
from random import randint
rand=randint(0, 9)
ts = time.time()
print(ts+rand)

1554738674.3973835


In [15]:
import time,os
import shutil, subprocess
from random import randint
os.chdir('/home/amunoz/Desktop/fiware-usage-control/ptp/docker/')
def execute_maven(self):
        prefix=str(randint(0, 9)+time.time())
        shutil.copytree('./cep', './cep'+prefix)
        mypath = './cep'+prefix
        os.chdir(mypath)
        p = subprocess.Popen(["mvn", "package"], stdout=subprocess.PIPE)
        output, err = p.communicate()
        print (output)
        return mypath
directory=execute_maven("")

/home/amunoz/Desktop/fiware-usage-control/ptp/docker
b"[\x1bINFO\x1b[m] Scanning for projects...\n[\x1bINFO\x1b[m] \n[\x1bINFO\x1b[m] \x1b------------------------------------------------------------------------\x1b[m\n[\x1bINFO\x1b[m] \x1bBuilding orion.flink.cep 1.0-SNAPSHOT\x1b[m\n[\x1bINFO\x1b[m] \x1b------------------------------------------------------------------------\x1b[m\n[\x1bINFO\x1b[m] \n[\x1bINFO\x1b[m] \x1b--- \x1bbuild-helper-maven-plugin:1.7:add-source\x1b[m \x1b(add-source)\x1b[m @ \x1borion.flink.cep\x1b ---\x1b[m\n[\x1bINFO\x1b[m] Source directory: /home/amunoz/Desktop/fiware-usage-control/ptp/docker/cep1554739418.8544347/src/main/scala added.\n[\x1bINFO\x1b[m] \n[\x1bINFO\x1b[m] \x1b--- \x1bmaven-resources-plugin:2.6:resources\x1b[m \x1b(default-resources)\x1b[m @ \x1borion.flink.cep\x1b ---\x1b[m\n[\x1bINFO\x1b[m] Using 'UTF-8' encoding to copy filtered resources.\n[\x1bINFO\x1b[m] Copying 1 resource\n[\x1bINFO\x1b[m] \n[\x1bINFO\x1b[m] \x1b--- \x1bmaven-compiler-

In [61]:
def run_job(self,jarId,flinkEndpoint,entryClass=''):
        if(entryClass==None or entryClass==''):
            entryClass="org.fiware.cosmos.orion.flink.cep.CEPMonitoring.class"
            #entry-class=org.fiware.cosmos.orion.flink.cep.CEPMonitoring.class&
        FLINK_ENDPOINT = "http://"+flinkEndpoint+"/jars/"+jarId+"/run?allowNonRestoredState=true"
        r = requests.post(url = FLINK_ENDPOINT) 
        pastebin_url = json.loads(r.text)
        jobID=pastebin_url.get("id")
        print("About the running Job:%s"%pastebin_url)    
        return jobID

run_job("","201986ff-9e15-4e3a-8a15-b62658a7d38f_orion.flink.cep-1.0-SNAPSHOT.jar","138.4.7.94:8082")

About the running Job:{'jobid': '4183f3da317b8abfab48332fa260a3c2'}


In [4]:

import requests

def delete_jar(self,jarId,flinkEndpoint):
        FLINK_ENDPOINT = "http://"+flinkEndpoint+"/jars/"+jarId
        r = requests.delete(url = FLINK_ENDPOINT)
        
delete_jar("","180c6db8-c01d-4064-b71e-0fab7b3eaf27_orion.flink.cep-1.0-SNAPSHOT.jar","138.4.7.94:8082")
